In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torch.optim.lr_scheduler import StepLR

from torch.utils.data import Dataset, DataLoader, random_split

import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

import copy
import time

from PIL import Image

In [2]:
# Loading Data
data_train = np.load('data_train.npy')
labels_train = np.load('labels_train.npy')

print(data_train.shape, labels_train.shape)

(270000, 3717) (3717,)


In [3]:
# Loading Unknown Class Data
data_unknown = np.load('data_unknown.npy')
labels_unknown = np.load('labels_unknown.npy')

print(data_unknown.shape, labels_unknown.shape)

(270000, 2000) (2000,)


In [4]:
data_train = np.concatenate((data_train, data_unknown),axis = 1)
labels_train = np.concatenate((labels_train, labels_unknown))
labels_train[labels_train == -1] = 10

In [5]:
str(int(labels_train[0]))

'6'

In [6]:
# Labels Integer Encoding

labels_names = ['Nike',
                'Adidas',
                'Ford',
                'Honda',
                'General Mills',
                'Unilever',
                "McDonald's",
                'KFC',
                'Gators',
                '3M',
                'Unknown']

In [7]:
corrected_labels_train = labels_train.copy()
changes = {
    0: [2400, 361, 2439, 1865, 2671, 893, 3625],
    1: [1288, 3033, 3037, 3599, 780, 3140, 3372],
    2: [1670, 673, 1995, 2804, 2769, 2714, 653],
    3: [2980, 2937, 603, 1819, 3704, 2025],
    4: [2665, 1649, 1777, 1563, 2986],
    5: [1587, 2750, 2763, 1917],
    6: [2015, 2388, 38, 401, 2818, 1875, 3675],
    7: [3154, 744, 452, 3283, 3412, 313],
    8: [556, 1213, 3253],
    9: [2940, 1314, 1108, 708, 3277, 728],
}
for new_label, indices_to_change in changes.items():
    for index in indices_to_change:
        corrected_labels_train[index] = new_label

In [8]:
# field names 
#fields = ['Name', 'Branch', 'Year', 'CGPA'] 
    
# data rows of csv file 
rows = []
for i in range(data_train.shape[1]):
    rows.append(["img"+str(i)+".png", str(int(corrected_labels_train[i]))])


In [9]:
class LogoDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        image = self.data[index].reshape(300, 300, 3)
        image = Image.fromarray(image)
        y_label = torch.tensor(int(self.labels[index]))

        if self.transform:
            image = self.transform(image)

        return (image, y_label)


In [10]:
# hyperparameters

in_channel = 3
num_classes = 11
learning_rate = 1e-4
batch_size = 32
num_epochs = 20

In [11]:
# Without Data Augmentation

train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [12]:
data_train = data_train.T

In [13]:
data_train, data_val, labels_train, labels_val = train_test_split(
    data_train, corrected_labels_train, test_size=0.2, random_state=0
)
train_dataset = LogoDataset(data=data_train, labels=labels_train, transform=train_transforms)
val_dataset = LogoDataset(data=data_val, labels=labels_val, transform=val_transforms)

In [14]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [16]:
# Model
model = models.googlenet(pretrained = True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.to(device)

/home/jiamin.chen/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jiamin.chen/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [17]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
scheduler = StepLR(optimizer, step_size=round(num_epochs/2), gamma=0.1)
# StepLR decays the learning rate by a factor (gamma) every few epochs (step_size)

In [18]:
# Early stopping
patience = 3
best_val_loss = float('inf')
epochs_without_improvement = 0

In [ ]:
# Train network

start = time.time()

# Initialize lists to store losses and accuracies
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    # Train
    model.train()
    train_loss = 0
    correct_train = 0
    total_train = 0

    for batch_idx, (data, targets) in enumerate(train_loader):
        if data is None or targets is None:
            continue

        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        train_loss += loss.item()
        _, predicted = scores.max(1)
        total_train += targets.size(0)
        correct_train += predicted.eq(targets).sum().item()

        # Backward
        optimizer.zero_grad()
        loss.backward()

        # Gradient descent or Adam step
        optimizer.step()
        

    scheduler.step()
    
    train_loss /= len(train_loader)
    train_accuracy = 100. * correct_train / total_train

    # Validate
    model.eval()
    val_loss = 0
    correct_val = 0
    total_val = 0
    
    with torch.no_grad():
        for batch_idx, (data, targets) in enumerate(val_loader):
            if data is None or targets is None:
                continue

            # Get data to cuda if possible
            data = data.to(device=device)
            targets = targets.to(device=device)

            # Forward
            scores = model(data)
            loss = criterion(scores, targets)
            
            val_loss += loss.item()
            _, predicted = scores.max(1)
            total_val += targets.size(0)
            correct_val += predicted.eq(targets).sum().item()
    
    val_loss /= len(val_loader)
    val_accuracy = 100. * correct_val / total_val

    # Append the losses and accuracies to the lists
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)

    # Print train and validation losses and accuracies
    print(f'Epoch {epoch}: Train Loss = {train_loss:.4f}, Train Accuracy = {train_accuracy:.2f}%, Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.2f}%')
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1

    if epochs_without_improvement >= patience:
        print(f"Early stopping: stopping training at epoch {epoch} due to no improvement in validation loss for {patience} epochs")
        break

torch.save(model.state_dict(), 'googlenet.pth')

end = time.time()
elapsed_time = end - start
print(f"Elapsed time: {elapsed_time} s")

Epoch 0: Train Loss = 0.8485, Train Accuracy = 80.91%, Validation Loss = 0.2532, Validation Accuracy = 93.79%
Epoch 1: Train Loss = 0.1719, Train Accuracy = 97.00%, Validation Loss = 0.1512, Validation Accuracy = 95.63%
Epoch 2: Train Loss = 0.0743, Train Accuracy = 98.67%, Validation Loss = 0.1203, Validation Accuracy = 96.42%
Epoch 3: Train Loss = 0.0350, Train Accuracy = 99.54%, Validation Loss = 0.1093, Validation Accuracy = 96.68%
Epoch 4: Train Loss = 0.0204, Train Accuracy = 99.80%, Validation Loss = 0.1010, Validation Accuracy = 97.12%
Epoch 5: Train Loss = 0.0143, Train Accuracy = 99.89%, Validation Loss = 0.0981, Validation Accuracy = 97.03%
Epoch 6: Train Loss = 0.0123, Train Accuracy = 99.91%, Validation Loss = 0.0977, Validation Accuracy = 96.94%
Epoch 7: Train Loss = 0.0123, Train Accuracy = 99.80%, Validation Loss = 0.0964, Validation Accuracy = 96.94%


In [ ]:
# Loss plot
plt.figure()
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.title("Loss vs Epoch")
plt.show()

# Accuracy plot
plt.figure()
plt.plot(train_accuracies, label="Train Accuracy")
plt.plot(val_accuracies, label="Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Accuracy vs Epoch")
plt.show()